In [1]:
import numpy as np
import pandas as pd

In [2]:
path = 'Recsys_data1.csv'

In [3]:
df = pd.read_csv(path)

In [4]:
def analysis(data: pd.DataFrame):
    data = data.drop(['banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0',  'coeff_sum1'], axis=1) #remove unnecessary cols (according to the task)
    print(data.head)
    print("Data contains null is ", data.isnull().values.any()) #check if contains null
    print(len(df[df['impressions'] == 0])) #check if impressions contains 0
    #count how many data on each day we have
    cur = data.copy()
    cur['date_time'] = cur['date_time'].apply(lambda x: x[:10])
    cur = cur.groupby(['date_time']).size()
    print(cur)
    return data

In [5]:
df = analysis(df)

<bound method NDFrame.head of                            date_time  zone_id  banner_id            oaid_hash  \
0         2021-09-27 00:01:30.000000        0          0  5664530014561852622   
1         2021-09-26 22:54:49.000000        1          1  5186611064559013950   
2         2021-09-26 23:57:20.000000        2          2  2215519569292448030   
3         2021-09-27 00:04:30.000000        3          3  6262169206735077204   
4         2021-09-27 00:06:21.000000        4          4  4778985830203613115   
...                              ...      ...        ...                  ...   
15821467  2021-10-02 15:51:35.000000      146        530  4329496688011613719   
15821468  2021-09-27 22:03:14.000000       12         22   453968700792456599   
15821469  2021-10-02 17:41:10.000000       12       1236  9112780675655118328   
15821470  2021-09-29 00:39:32.000000      967         21  6968514095695555037   
15821471  2021-09-28 07:00:18.000000       19        635  8754492963501134426  

Выводы из анализа данных:

1. В данных нет null.
2. Все impressions равны 1 (что логично), так что уберем этот столбец.
3. Столбец campaign_clicks содержит какие-то странные данные, непонятно, что они нам дают. Как будто бы мы не можем сделать никакой вывод, зная только о том, что человеку уже показывалась похожая реклама. Так что уберем его, так и данные поменьше станут.

4. Также я не буду использовать дату при работе. Само число дня недели использовать точно странно, ведь их у нас мало, да и на тесте дата будет отличаться, так что ничему хорошему у модели научиться не получится. Можно было бы использовать время дня, что могло бы помочь (например, вечером люди могут быть уставшие и не кликать на рекламу или наоборот днем быть занятыми и так далее). Но этот момент может быть подвержен сезонности (например в выходные люди могут вести себя не так, как на рабочей неделе), а данных у нас опять же слишком мало, для учета таких вещей.

5. Также есть день 2021-09-01, для которого есть всего одна запись и он далеко от остальных. Видимо он попал в данные случайно, его надо удалить.

6. Последний день (который будем брать в тест) 2021-10-02, предпоследний 20-10-01 берем в валидационное множество.

7. Остальные данные категориальные

Ровно этим и будем заниматься в функции feature_engineering. 


Я пробовала пытаться как-то объединять в филды оставшиеся данные, но в итоге так и оставила для каждого параметра свой филд, потому что так было лучше.


In [12]:
from sklearn.preprocessing import OrdinalEncoder

#feature engineering according to the analysis and spliting into train and test
def feature_engineering(data: pd.DataFrame) -> pd.DataFrame:
    data = data.drop(['impressions', 'campaign_clicks'], axis=1)
    data = data[data['date_time'] > '2021-09-20']
    
    cur = data[data['date_time'] < '2021-10-01']
    x_train = cur.drop(['clicks', 'date_time'], axis=1)
    y_train = cur['clicks']
    
    cur = data[(data['date_time'] < '2021-10-02') & (data['date_time'] >= '2021-10-01')] 
    x_val = cur.drop(['clicks', 'date_time'], axis=1)
    y_val = cur['clicks']
    
    cur = data[data['date_time'] >= '2021-10-02']
    x_test = cur.drop(['clicks', 'date_time'], axis=1)
    y_test = cur['clicks']
    
    return x_train, y_train, x_val, y_val, x_test, y_test

In [13]:
x_train, y_train, x_val, y_val, x_test, y_test = feature_engineering(df)
df = []

In [20]:
#https://github.com/Bobe24/Dataframe2libffm/tree/3e34cb0c195242560d85753b2963ad845691e14e
category_column = ['oaid_hash', 'banner_id', 'clicks', 'zone_id', 'os_id', 'country_id']

class FFMFormat:
    def __init__(self):
        self.field_index_ = None
        self.feature_index_ = None
        self.y = None

    def fit(self, df, y=None):
        self.y = y
        df_ffm = df[df.columns.difference([self.y])]
        if self.field_index_ is None:
            self.field_index_ = {col: i for i, col in enumerate(df_ffm)}

        if self.feature_index_ is not None:
            last_idx = max(list(self.feature_index_.values()))

        if self.feature_index_ is None:
            self.feature_index_ = dict()
            last_idx = 0

        for col in df_ffm.columns:
            vals = df_ffm[col].unique()
            for val in vals:
                if pd.isnull(val):
                    continue
                name = '{}_{}'.format(col, val)
                if name not in self.feature_index_:
                    self.feature_index_[name] = last_idx
                    last_idx += 1
            self.feature_index_[col] = last_idx
            last_idx += 1
        return self

    def fit_transform(self, df, y=None):
        self.fit(df, y)
        return self.transform(df)

    def transform_row_(self, row, t):
        ffm = []
        if self.y != None:
            ffm.append(str(row.loc[row.index == self.y][0]))
        if self.y is None:
            ffm.append(str(0))

        for col, val in row.loc[row.index != self.y].to_dict().items():
            col_type = t[col]
            name = '{}_{}'.format(col, val)
            # if col_type.kind == 'O':
            if col in category_column:
                ffm.append('{}:{}:1'.format(self.field_index_[col],
                                            self.feature_index_[name]))
            else:
            # elif col_type.kind == 'i':
                ffm.append('{}:{}:{}'.format(self.field_index_[col],
                                             self.feature_index_[col], val))
        return ' '.join(ffm)

    def transform(self, df):
        t = df.dtypes.to_dict()
        return pd.Series(
            {idx: self.transform_row_(row, t) for idx, row in df.iterrows()})

Переведем данные в libffm формат с помощью алогритма выше. Почему-то при записи результата в csv файл пишется лишний первый 0. Чтобы дальше использовать файлы в модели, нужно удалить ноль руками из файла, иначе модель упадет
:(

In [21]:
train_df = x_train
train_df['clicks'] = y_train

val_df = x_val
val_df['clicks'] = y_val

test_df = x_test
test_df['clicks'] = y_test

In [22]:
ffm_encoder = FFMFormat()
ffm_train_data = ffm_encoder.fit_transform(train_df, y='clicks')
ffm_train_data.to_csv('train_ffm.txt', index=False)

In [23]:
ffm_val_data = ffm_encoder.fit_transform(val_df, y='clicks')
ffm_val_data.to_csv('val_ffm.txt', index=False)

Теперь перейдем к самой модели. Для работы я выбрала библиотеку xlearn, потому что она достаточно удобная и быстрее альтернатив, которые я видела.

In [41]:
!pip install xlearn

In [24]:
import xlearn as xl

Запустим модель с разными параметрами. Я еще запускала с другими lr, с ними результат был гораздо хуже. Так что в целях экономии времени запуска не добавила сюда эти эксперименты.

In [28]:
from sklearn.model_selection import ParameterGrid

params_grid = {
    'task': ['binary'],
    'lr': [0.1],
    'lambda':[0.1, 0.002, 0.0002],
    'k': [4, 8], 
    'metric': ['auc'], 
    'epoch': [30]
}

for params in ParameterGrid(params_grid):
    print(params)
    ffm_model = xl.create_ffm()
    ffm_model.setTrain("train_ffm.txt")
    ffm_model.setValidate("val_ffm.txt")
    ffm_model.fit(params, "model.out")

{'epoch': 30, 'k': 4, 'lambda': 0.1, 'lr': 0.1, 'metric': 'auc', 'task': 'binary'}
----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[------------] xLearn uses 12 threads for training task.
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (train_ffm.txt.bin) found. Skip converting text to binary.
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (val_ffm.txt.bin) found. Skip converting text to binary.
[------------] Number of Feature: 5665369
[------------] Number of 

[------------] Model file: model.out
[------------] Time cost for saving model: 1.34 (sec)
[ ACTION     ] Finish training
[ ACTION     ] Clear the xLearn environment ...
{'epoch': 30, 'k': 4, 'lambda': 0.0002, 'lr': 0.1, 'metric': 'auc', 'task': 'binary'}
[------------] Total time cost: 125.42 (sec)
----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[------------] xLearn uses 12 threads for training task.
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (train_ffm.txt.bin) found. Skip converting text to binary.
[------------] First c

[ ACTION     ] Start to save model ...
[------------] Model file: model.out
[------------] Time cost for saving model: 2.54 (sec)
[ ACTION     ] Finish training
[ ACTION     ] Clear the xLearn environment ...
[------------] Total time cost: 136.36 (sec)
{'epoch': 30, 'k': 8, 'lambda': 0.002, 'lr': 0.1, 'metric': 'auc', 'task': 'binary'}
----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[------------] xLearn uses 12 threads for training task.
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (train_ffm.txt.bin) found. Skip converting 

[ ACTION     ] Start to save model ...
[------------] Model file: model.out
[------------] Time cost for saving model: 2.60 (sec)
[ ACTION     ] Finish training
[ ACTION     ] Clear the xLearn environment ...
[------------] Total time cost: 60.12 (sec)


Видим, что $k=8$ немного обыгрывает $k=4$. А еще что лучший результат достигается на граничном значении $lambda = 0.0002$. Попробуем еще уменьшить его, вдруг станет лучше.

In [29]:
params_grid = {
    'task': ['binary'],
    'lr': [0.1],
    'lambda':[0.0002, 0.00002, 0.000002],
    'k': [8], 
    'metric': ['auc'], 
    'epoch': [30]
}

for params in ParameterGrid(params_grid):
    print(params)
    ffm_model = xl.create_ffm()
    ffm_model.setTrain("train_ffm.txt")
    ffm_model.setValidate("val_ffm.txt")
    ffm_model.fit(params, "model.out")

{'epoch': 30, 'k': 8, 'lambda': 0.0002, 'lr': 0.1, 'metric': 'auc', 'task': 'binary'}
----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[------------] xLearn uses 12 threads for training task.
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (train_ffm.txt.bin) found. Skip converting text to binary.
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (val_ffm.txt.bin) found. Skip converting text to binary.
[------------] Number of Feature: 5665369
[------------] Number 

[------------] Total time cost: 39.57 (sec)


Здесь заметной разницы между результатами нет. Выберем уже знакомый нам с $lambda=0.0002$ - он немного лучше по auc, но немного проигрывает по log_loss. И теперь запустим лучшую модель на тесте.

In [30]:
ffm_test_data = ffm_encoder.fit_transform(test_df, y='clicks')
ffm_test_data.to_csv('test_ffm.txt', index=False)

In [31]:
ffm_model = xl.create_ffm()
ffm_model.setTrain("train_ffm.txt")
params = {
    'task': 'binary',
    'lr': 0.1,
    'lambda':0.0002,
    'k':8, 
    'metric':'auc', 
    'epoch':30
}
ffm_model.setValidate("val_ffm.txt")

In [32]:
ffm_model.fit(params, "model.out")

----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[------------] xLearn uses 12 threads for training task.
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (train_ffm.txt.bin) found. Skip converting text to binary.
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (val_ffm.txt.bin) found. Skip converting text to binary.
[------------] Number of Feature: 5665369
[------------] Number of Field: 5
[------------] Time cost for reading problem: 12.30 (sec)
[ ACTION     ] I

In [36]:
ffm_model.setTest('test_ffm.txt')
ffm_model.setSigmoid()
ffm_model.predict("model.out", "output.txt")

----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[------------] xLearn uses 12 threads for prediction task.
[ ACTION     ] Load model ...
[------------] Load model from model.out
[------------] Loss function: cross-entropy
[------------] Score function: ffm
[------------] Number of Feature: 5665369
[------------] Number of K: 8
[------------] Number of field: 5
[------------] Time cost for loading model: 1.10 (sec)
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (test_ffm.txt.bin) found. Skip converting text to binary.
[----------

In [43]:
import pandas as pd
y_pred = pd.read_csv('output.txt', header=None)

In [44]:
from sklearn.metrics import roc_auc_score, log_loss
auc_metric = roc_auc_score(y_test, y_pred)
log_loss_metric = log_loss(y_test, y_pred)
print('Auc: {:.3f}, log_loss: {:.3f}'.format(auc_metric, log_loss_metric))


Auc: 0.791, log_loss: 0.136


В линейной модели у меня получились метрики auc: 0.779, log_loss: 0.134.
Получается, что ffm немного выиграла по auc, но совсем чуть-чуть проиграла по лоссу. Так как я выбрала модель с оптимальным auc, может, если бы мы взяли лямбду еще меньше (то есть выбрали оптимальную по лоссу), то не проиграли бы по лоссу. Ну или это просто случайность:)